In [1]:
import pandas as pd
import numpy as np

In [2]:
logsTrocafone = pd.read_csv('Data/events_up_to_01062018.csv', low_memory = False)

#trainingSet = pd.read_csv('trainingSet.csv')
#trainingSet = trainingSet.set_index('person')

#testingSet = pd.read_csv('testingSet.csv')
#testingSet = testingSet.set_index('person')

#trainingSet.head()

In [3]:
df = logsTrocafone

In [4]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

Ordeno por persona y tiempo

In [5]:
df.sort_values(['person', 'timestamp'], ascending=[True, True], inplace = True)

Paso los caracteres a minúsculas

In [7]:
df['search_term'] = df['search_term'].str.lower()
df['model'] = df['model'].str.lower()

Elimino símbolos que no aportan nada al análisis

In [8]:
df['search_term'] = df['search_term'].str.replace('!','').str.replace('?','').str.replace('¡','').str.replace('¿','').str.replace('+','').str.replace('-','').str.replace('*','').str.replace('%','').str.replace(',','').str.replace(';','').str.replace('.','').str.replace(':','').str.replace('_','').str.replace('\'','').str.replace('\"','').str.replace('\\','').str.replace('/','').str.replace('(','').str.replace(')','').str.replace('{','').str.replace('}','').str.replace('[','').str.replace(']','').str.replace('@','').str.replace('º','').str.replace('&','')

In [9]:
dfo = df

Creo la columna 'search' para identificar las filas relacionadas a cada busqueda

In [10]:
df = dfo
df['timediff'] = df['timestamp'].diff()
df['timediff'] = df['timediff'].dt.total_seconds()

df['newperson']= df['person'].apply(lambda x: int(x, 16))
df['newperson'] = df['newperson'].diff()
df['prueba'] = df['newperson']
df['newperson'] = ((df['newperson'] != 0.0) | (df.index == 0))

maxtime = 16368
df['newentry'] = ((df['timediff'] > maxtime) | (df['newperson']))
#df['entry'] = df['newentry'].cumsum()
df['newsearch'] = ((df['event'] == 'searched products') | (df['newentry']))
df['validsearch'] = ((df['event'] == 'searched products') & (df['search_term']))
df['search'] = df['newsearch'].cumsum()

In [11]:
print(df.loc[:40, ['person', 'search', 'prueba', 'newperson', 'newentry', 'newsearch', 'validsearch']])

           person  search   prueba  newperson  newentry  newsearch  \
1507286  0008ed71       1      NaN       True      True       True   
2336760  0008ed71       1      0.0      False     False      False   
1507716  0008ed71       1      0.0      False     False      False   
2336761  0008ed71       1      0.0      False     False      False   
2122051  0008ed71       1      0.0      False     False      False   
1505383  0008ed71       1      0.0      False     False      False   
2146920  00091926       2  11189.0       True      True       True   
121981   00091926       2      0.0      False     False      False   
120337   00091926       2      0.0      False     False      False   
120458   00091926       2      0.0      False     False      False   
120417   00091926       2      0.0      False     False      False   
120338   00091926       2      0.0      False     False      False   
120418   00091926       2      0.0      False     False      False   
83382    00091926   

Filtro las filas relacionadas a las búsquedas

In [12]:
df = df.groupby('search').filter(lambda x: x.iloc[0]['validsearch'])

In [13]:
df_filtered = df

In [15]:
df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,screen_resolution,operating_system_version,browser_version,timediff,newperson,prueba,newentry,newsearch,validsearch,search
61644,2018-05-29 00:29:03,searched products,000c79fe,NaN,NaN,NaN,NaN,NaN,NaN,"9931,9944,9902,9988,10084,9973,10322,10254,158...",...,NaN,NaN,NaN,59.0,False,0.0,False,True,True,30
61803,2018-05-29 00:29:39,viewed product,000c79fe,NaN,9944.0,iphone 7,Bom,128GB,Preto Matte,NaN,...,NaN,NaN,NaN,36.0,False,0.0,False,False,False,30
62961,2018-05-29 00:29:51,checkout,000c79fe,NaN,9944.0,iphone 7,Bom,128GB,Preto Matte,NaN,...,NaN,NaN,NaN,12.0,False,0.0,False,False,False,30
57876,2018-05-29 00:35:47,viewed product,000c79fe,NaN,9944.0,iphone 7,Bom,128GB,Preto Matte,NaN,...,NaN,NaN,NaN,356.0,False,0.0,False,False,False,30
57877,2018-05-29 00:35:48,searched products,000c79fe,NaN,NaN,NaN,NaN,NaN,NaN,"9931,9944,9902,9988,10084,9973,10322,10254,158...",...,NaN,NaN,NaN,1.0,False,0.0,False,True,True,31


Creo un DataFrame con una columna llamada 'busquedas' que contiene la marca buscada a partir del evento 'searched products' hasta otra busqueda o el fin de la sesión.  
Le agrego la columna 'nmodel' que contiene la marca que figura en 'model' de modo de poder comparar con 'busquedas'  
Agrego las columnas siguientes que pueden tomar como valor a 0 o 1:  
**bcm**: Busqueda con marca: es 1 si habia una marca en los términos de búsqueda  
**vam**: Visitas a marca: es 1 si tras la búsqueda de una marca se visita un producto de la misma  
**vao**: Visitas a otra marca: es 1 si tras la búsqueda de una marca se visita un producto de otra marca  
**coam**: Checkouts a marca: es 1 si tras la búsqueda de una marca se hace un checkout con un producto de la misma  
**coao**: Checkouts a otra marca: es 1 si tras la búsqueda de una marca se hace un checkout con un producto de otra marca  
**cam**: Conversión a marca: es 1 si tras la búsqueda de una marca se hace una conversion con un producto de la misma  
**cao**: Conversión a otra marca: es 1 si tras la búsqueda de una marca se hace una conversion con un producto de otra marca  
**lam**: Lead a marca: es 1 si tras la búsqueda de una marca se hace un lead con un producto de la misma  
**lao**: Lead a otra marca: es 1 si tras la búsqueda de una marca se hace un lead con un producto de otra marca

In [16]:
df = df_filtered
df2 = df

def contenido(s,x):
    return x.find(s) >= 0

def obtener_marca(x):
    if (type(x) == float):
        return np.nan
    elif (contenido('asus',x) or contenido('live',x) or contenido('zenfone',x)):
        return 'asus'
    elif (contenido('blackberry',x)):
        return 'blackberry'
    elif (contenido('lg',x)):
        return 'lg'
    elif (contenido('lenovo',x) or contenido('vibe',x)):
        return 'lenovo'
    elif (contenido('moto',x)):
        return 'motorola'
    elif (contenido('quantum',x)):
        return 'quantum'
    elif (contenido('samsung',x) or contenido('galaxy',x)):
        return 'samsung'
    elif (contenido('sony',x) or contenido('xperia',x)):
        return 'sony'
    elif (contenido('apple',x) or contenido('iphone',x) or contenido('ipad',x)):
        return 'apple'
    else:
        return np.nan

def obtener_marca_de_busqueda(x):
    return obtener_marca(x.iloc[0])

dfg = df.groupby('search', as_index=False)
df2['busquedas'] = dfg['search_term'].transform(obtener_marca_de_busqueda)
df2['nmodel'] = df2.apply(lambda x: obtener_marca(x['model']), axis=1)

df2['bcm'] = ((df2['event'] == 'searched products') & (df2['busquedas'])).astype(int)
df2['vam'] = ((df2['event'] == 'viewed product') & (df2['busquedas']) & (df2['busquedas'] == df2['nmodel'])).astype(int)
df2['vao'] = ((df2['event'] == 'viewed product') & (df2['busquedas']) & (df2['busquedas'] != df2['nmodel'])).astype(int)
df2['coam'] = ((df2['event'] == 'checkout') & (df2['busquedas']) & (df2['busquedas'] == df2['nmodel'])).astype(int)
df2['coao'] = ((df2['event'] == 'checkout') & (df2['busquedas']) & (df2['busquedas'] != df2['nmodel'])).astype(int)
df2['cam'] = ((df2['event'] == 'conversion') & (df2['busquedas']) & (df2['busquedas'] == df2['nmodel'])).astype(int)
df2['cao'] = ((df2['event'] == 'conversion') & (df2['busquedas']) & (df2['busquedas'] != df2['nmodel'])).astype(int)
df2['lam'] = ((df2['event'] == 'lead') & (df2['busquedas']) & (df2['busquedas'] == df2['nmodel'])).astype(int)
df2['lao'] = ((df2['event'] == 'lead') & (df2['busquedas']) & (df2['busquedas'] != df2['nmodel'])).astype(int)

print(df2.loc[:1000, ['search', 'event', 'search_term', 'model', 'nmodel', 'busquedas', 'bcm', 'vam', 'vao', 'coam', 'coao', 'cam', 'cao', 'lam', 'lao']])

/home/sebastian/.local/lib/python2.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sebastian/.local/lib/python2.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sebastian/.local/lib/python2.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

         search              event search_term                         model  \
61644        30  searched products    iphone 7                           NaN   
61803        30     viewed product         NaN                      iphone 7   
62961        30           checkout         NaN                      iphone 7   
57876        30     viewed product         NaN                      iphone 7   
57877        31  searched products    iphone 7                           NaN   
57880        31     viewed product         NaN                      iphone 7   
63117        32  searched products   iphone 7s                           NaN   
63122        33  searched products    iphone x                           NaN   
63278        34  searched products    iphone 6                           NaN   
63280        35  searched products   iphone 6s                           NaN   
57884        36  searched products   galaxy a8                           NaN   
62823        37  searched products   gal

In [18]:
print(df2.loc[df2['person'] == '15ea8012', ['person', 'search', 'event', 'nmodel', 'busquedas', 'bcm', 'vam', 'vao', 'coam', 'coao', 'cam', 'cao', 'lam', 'lao']])

           person  search              event   nmodel busquedas  bcm  vam  \
262269   15ea8012   24467  searched products      NaN   samsung    1    0   
262238   15ea8012   24467     viewed product  samsung   samsung    0    1   
261192   15ea8012   24467     viewed product  samsung   samsung    0    1   
262272   15ea8012   24467     viewed product  samsung   samsung    0    1   
262567   15ea8012   24467     viewed product  samsung   samsung    0    1   
262241   15ea8012   24467     viewed product  samsung   samsung    0    1   
262576   15ea8012   24467     viewed product  samsung   samsung    0    1   
243424   15ea8012   24467         staticpage      NaN   samsung    0    0   
261749   15ea8012   24468  searched products      NaN   samsung    1    0   
200166   15ea8012   24470  searched products      NaN   samsung    1    0   
66       15ea8012   24470     viewed product  samsung   samsung    0    1   
200165   15ea8012   24470     viewed product  samsung   samsung    0    1   

### A partir del df2 creo los features

In [19]:
dftmp = df2[(df2[['bcm']] == 1).all(axis=1)]
dftmp = dftmp.loc[:, ['person', 'busquedas', 'bcm']]
print(dftmp.loc[dftmp['person'] == 'ff77d8a0', :])

          person busquedas  bcm
220563  ff77d8a0     apple    1
220750  ff77d8a0     apple    1
220715  ff77d8a0     apple    1
182926  ff77d8a0     apple    1
223108  ff77d8a0     apple    1
220713  ff77d8a0     apple    1
183065  ff77d8a0     apple    1
220773  ff77d8a0     apple    1
220774  ff77d8a0     apple    1
183220  ff77d8a0     apple    1
221944  ff77d8a0  motorola    1
221943  ff77d8a0  motorola    1
221549  ff77d8a0  motorola    1
141127  ff77d8a0     apple    1
141128  ff77d8a0     apple    1
74074   ff77d8a0     apple    1
140535  ff77d8a0     apple    1
74173   ff77d8a0     apple    1
74265   ff77d8a0     apple    1
115127  ff77d8a0     apple    1
119379  ff77d8a0     apple    1
115492  ff77d8a0     apple    1
119396  ff77d8a0     apple    1
118531  ff77d8a0     apple    1
76283   ff77d8a0     apple    1
142403  ff77d8a0     apple    1
142277  ff77d8a0     apple    1
141077  ff77d8a0     apple    1
142294  ff77d8a0     apple    1
142295  ff77d8a0     apple    1
142057  

In [20]:
dftmp = dftmp.pivot_table(index='person',columns='busquedas',aggfunc=sum)
dftmp.fillna(0,inplace=True)
dftmp.columns = dftmp.columns.droplevel()
dftmp.columns.name = None
dftmp.reset_index(inplace=True)

In [21]:
dftmp['person'].is_unique

True

In [22]:
dftmp = dftmp.set_index('person')

In [23]:
dftmp

,apple,asus,blackberry,lenovo,lg,motorola,quantum,samsung,sony
person,,,,,,,,,
000c79fe,6.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
000e619d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
001001be,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001802e4,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0019e639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
0027574e,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002aea56,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002e74b2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002ed810,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
trainingSet = pd.read_csv('trainingSet.csv')
trainingSet = trainingSet.set_index('person')

testingSet = pd.read_csv('testingSet.csv')
testingSet = testingSet.set_index('person')

trainingSet.head()

,CantidadEventos,CantidadCompras,CantidadLeads,VentasHaceUnMes,LeadsHaceUnMes,VentasHaceDosMeses,LeadsHaceDosMeses,CantidadCheckouts,VentasMes1,VentasMes2,...,EntraronConComputadora,EntraronConSmartphone,EntraronConTablet,CantidadAdHits,DiasDesdeUltimoAdHit,DiasDesdeUltimaBusqueda,CantidadDeColoresVistos,CantidadDiasDiferentesDeVisita,CantidadDeModelosVistos,EsDeBrazil
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,6,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1000.0,1000.0,3,1,3,1.0
000c79fe,17,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,3.0,3.0,1,1,1,1.0
001802e4,19,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,5.0,1.0,1.0,2,1,2,1.0
0019e639,471,2.0,0.0,1.0,0.0,1.0,0.0,15.0,0.0,1.0,...,1.0,0.0,0.0,29.0,10.0,10.0,14,14,26,1.0
001b0bf9,7,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1000.0,1000.0,2,1,1,0.0


In [25]:
nuevoTrainingSet = trainingSet.merge(dftmp, how='left', left_index=True, right_index=True)

nuevoTrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19414 entries, 0008ed71 to fff78145
Data columns (total 38 columns):
CantidadEventos                   19414 non-null int64
CantidadCompras                   19414 non-null float64
CantidadLeads                     19414 non-null float64
VentasHaceUnMes                   19414 non-null float64
LeadsHaceUnMes                    19414 non-null float64
VentasHaceDosMeses                19414 non-null float64
LeadsHaceDosMeses                 19414 non-null float64
CantidadCheckouts                 19414 non-null float64
VentasMes1                        19414 non-null float64
VentasMes2                        19414 non-null float64
VentasMes3                        19414 non-null float64
VentasMes4                        19414 non-null float64
PromedioVentasPorMes              19414 non-null float64
DiasDesdeUltimoEvento             19414 non-null float64
DiasDesdePrimerEvento             19414 non-null float64
DiasDesdeUltimaConversion        

In [26]:
nuevoTrainingSet['apple'].fillna(0, inplace=True)
nuevoTrainingSet['asus'].fillna(0, inplace=True)
nuevoTrainingSet['blackberry'].fillna(0, inplace=True)
nuevoTrainingSet['lenovo'].fillna(0, inplace=True)
nuevoTrainingSet['lg'].fillna(0, inplace=True)
nuevoTrainingSet['motorola'].fillna(0, inplace=True)
nuevoTrainingSet['quantum'].fillna(0, inplace=True)
nuevoTrainingSet['samsung'].fillna(0, inplace=True)
nuevoTrainingSet['sony'].fillna(0, inplace=True)

nuevoTrainingSet['apple'] = nuevoTrainingSet['apple'].astype(int)
nuevoTrainingSet['asus'] = nuevoTrainingSet['asus'].astype(int)
nuevoTrainingSet['blackberry'] = nuevoTrainingSet['blackberry'].astype(int)
nuevoTrainingSet['lenovo'] = nuevoTrainingSet['lenovo'].astype(int)
nuevoTrainingSet['lg'] = nuevoTrainingSet['lg'].astype(int)
nuevoTrainingSet['motorola'] = nuevoTrainingSet['motorola'].astype(int)
nuevoTrainingSet['quantum'] = nuevoTrainingSet['quantum'].astype(int)
nuevoTrainingSet['samsung'] = nuevoTrainingSet['samsung'].astype(int)
nuevoTrainingSet['sony'] = nuevoTrainingSet['sony'].astype(int)
nuevoTrainingSet.head()

,CantidadEventos,CantidadCompras,CantidadLeads,VentasHaceUnMes,LeadsHaceUnMes,VentasHaceDosMeses,LeadsHaceDosMeses,CantidadCheckouts,VentasMes1,VentasMes2,...,EsDeBrazil,apple,asus,blackberry,lenovo,lg,motorola,quantum,samsung,sony
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,6,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,1.0,0,0,0,0,0,0,0,0,0
000c79fe,17,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,6,0,0,0,0,0,0,2,0
001802e4,19,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,2,0,0,0,0,0,0,0,0
0019e639,471,2.0,0.0,1.0,0.0,1.0,0.0,15.0,0.0,1.0,...,1.0,0,0,0,0,0,0,0,3,0
001b0bf9,7,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,0


In [27]:
nuevoTestingSet = testingSet.merge(dftmp, how='left', left_index=True, right_index=True)

nuevoTestingSet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19415 entries, 00091926 to fffd1246
Data columns (total 38 columns):
CantidadEventos                   19415 non-null int64
CantidadCompras                   19415 non-null float64
CantidadLeads                     19415 non-null float64
VentasHaceUnMes                   19415 non-null float64
LeadsHaceUnMes                    19415 non-null float64
VentasHaceDosMeses                19415 non-null float64
LeadsHaceDosMeses                 19415 non-null float64
CantidadCheckouts                 19415 non-null float64
VentasMes1                        19415 non-null float64
VentasMes2                        19415 non-null float64
VentasMes3                        19415 non-null float64
VentasMes4                        19415 non-null float64
PromedioVentasPorMes              19415 non-null float64
DiasDesdeUltimoEvento             19415 non-null float64
DiasDesdePrimerEvento             19415 non-null float64
DiasDesdeUltimaConversion        

In [28]:
nuevoTestingSet['apple'].fillna(0, inplace=True)
nuevoTestingSet['asus'].fillna(0, inplace=True)
nuevoTestingSet['blackberry'].fillna(0, inplace=True)
nuevoTestingSet['lenovo'].fillna(0, inplace=True)
nuevoTestingSet['lg'].fillna(0, inplace=True)
nuevoTestingSet['motorola'].fillna(0, inplace=True)
nuevoTestingSet['quantum'].fillna(0, inplace=True)
nuevoTestingSet['samsung'].fillna(0, inplace=True)
nuevoTestingSet['sony'].fillna(0, inplace=True)

nuevoTestingSet['apple'] = nuevoTestingSet['apple'].astype(int)
nuevoTestingSet['asus'] = nuevoTestingSet['asus'].astype(int)
nuevoTestingSet['blackberry'] = nuevoTestingSet['blackberry'].astype(int)
nuevoTestingSet['lenovo'] = nuevoTestingSet['lenovo'].astype(int)
nuevoTestingSet['lg'] = nuevoTestingSet['lg'].astype(int)
nuevoTestingSet['motorola'] = nuevoTestingSet['motorola'].astype(int)
nuevoTestingSet['quantum'] = nuevoTestingSet['quantum'].astype(int)
nuevoTestingSet['samsung'] = nuevoTestingSet['samsung'].astype(int)
nuevoTestingSet['sony'] = nuevoTestingSet['sony'].astype(int)
nuevoTestingSet.head()

,CantidadEventos,CantidadCompras,CantidadLeads,VentasHaceUnMes,LeadsHaceUnMes,VentasHaceDosMeses,LeadsHaceDosMeses,CantidadCheckouts,VentasMes1,VentasMes2,...,EsDeBrazil,apple,asus,blackberry,lenovo,lg,motorola,quantum,samsung,sony
person,,,,,,,,,,,,,,,,,,,,,
00091926,448,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,1.0,0,0,0,0,0,0,0,0,0
00091a7a,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0,0,0,0,0,0,0,0,0
000ba417,206,1.0,0.0,1.0,0.0,1.0,0.0,6.0,0.0,0.0,...,1.0,0,0,0,0,0,0,0,0,0
000e4d9e,411,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0,0,0,0,0,0,0,0,0
000e619d,68,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0,0,0,0,0,0,0,1,0


In [100]:
#nuevoTrainingSet.to_csv('trainingSet2.csv')
#Este comando reemplaza el training set existente con la feature agregada

In [101]:
#nuevoTestingSet.to_csv('testingSet2.csv')
#Este comando reemplaza el testing set existente con la feature agregada

In [29]:
dftmp = df2.loc[:, ['person', 'bcm', 'vam', 'vao', 'coam', 'coao', 'cam', 'cao', 'lam', 'lao']]
dftmp.head()

,person,bcm,vam,vao,coam,coao,cam,cao,lam,lao
61644,000c79fe,1,0,0,0,0,0,0,0,0
61803,000c79fe,0,1,0,0,0,0,0,0,0
62961,000c79fe,0,0,0,1,0,0,0,0,0
57876,000c79fe,0,1,0,0,0,0,0,0,0
57877,000c79fe,1,0,0,0,0,0,0,0,0


In [30]:
dftmp = dftmp.groupby('person', as_index=True).sum()
dftmp.head()

,bcm,vam,vao,coam,coao,cam,cao,lam,lao
person,,,,,,,,,
000c79fe,8,3,0,1,0,0,0,0,0
000e619d,1,2,10,0,0,0,0,0,0
001001be,12,36,0,3,0,1,0,0,0
001802e4,2,0,0,0,0,0,0,0,0
0019e639,3,9,0,2,0,0,0,0,0


In [31]:
nuevoTrainingSet = nuevoTrainingSet.merge(dftmp, how='left', left_index=True, right_index=True)

nuevoTrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19414 entries, 0008ed71 to fff78145
Data columns (total 47 columns):
CantidadEventos                   19414 non-null int64
CantidadCompras                   19414 non-null float64
CantidadLeads                     19414 non-null float64
VentasHaceUnMes                   19414 non-null float64
LeadsHaceUnMes                    19414 non-null float64
VentasHaceDosMeses                19414 non-null float64
LeadsHaceDosMeses                 19414 non-null float64
CantidadCheckouts                 19414 non-null float64
VentasMes1                        19414 non-null float64
VentasMes2                        19414 non-null float64
VentasMes3                        19414 non-null float64
VentasMes4                        19414 non-null float64
PromedioVentasPorMes              19414 non-null float64
DiasDesdeUltimoEvento             19414 non-null float64
DiasDesdePrimerEvento             19414 non-null float64
DiasDesdeUltimaConversion        

In [32]:
nuevoTrainingSet = nuevoTrainingSet.fillna(0)
nuevoTrainingSet.head()

,CantidadEventos,CantidadCompras,CantidadLeads,VentasHaceUnMes,LeadsHaceUnMes,VentasHaceDosMeses,LeadsHaceDosMeses,CantidadCheckouts,VentasMes1,VentasMes2,...,sony,bcm,vam,vao,coam,coao,cam,cao,lam,lao
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,6,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000c79fe,17,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,8.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
001802e4,19,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0019e639,471,2.0,0.0,1.0,0.0,1.0,0.0,15.0,0.0,1.0,...,0,3.0,9.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
001b0bf9,7,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
nuevoTestingSet = nuevoTestingSet.merge(dftmp, how='left', left_index=True, right_index=True)

nuevoTestingSet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19415 entries, 00091926 to fffd1246
Data columns (total 47 columns):
CantidadEventos                   19415 non-null int64
CantidadCompras                   19415 non-null float64
CantidadLeads                     19415 non-null float64
VentasHaceUnMes                   19415 non-null float64
LeadsHaceUnMes                    19415 non-null float64
VentasHaceDosMeses                19415 non-null float64
LeadsHaceDosMeses                 19415 non-null float64
CantidadCheckouts                 19415 non-null float64
VentasMes1                        19415 non-null float64
VentasMes2                        19415 non-null float64
VentasMes3                        19415 non-null float64
VentasMes4                        19415 non-null float64
PromedioVentasPorMes              19415 non-null float64
DiasDesdeUltimoEvento             19415 non-null float64
DiasDesdePrimerEvento             19415 non-null float64
DiasDesdeUltimaConversion        

In [34]:
nuevoTestingSet = nuevoTestingSet.fillna(0)
nuevoTestingSet.head()

,CantidadEventos,CantidadCompras,CantidadLeads,VentasHaceUnMes,LeadsHaceUnMes,VentasHaceDosMeses,LeadsHaceDosMeses,CantidadCheckouts,VentasMes1,VentasMes2,...,sony,bcm,vam,vao,coam,coao,cam,cao,lam,lao
person,,,,,,,,,,,,,,,,,,,,,
00091926,448,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00091a7a,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000ba417,206,1.0,0.0,1.0,0.0,1.0,0.0,6.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000e4d9e,411,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000e619d,68,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,1.0,2.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
nuevoTrainingSet.to_csv('trainingSet2.csv')
#Este comando reemplaza el training set existente con la feature agregada

In [36]:
nuevoTestingSet.to_csv('testingSet2.csv')
#Este comando reemplaza el testing set existente con la feature agregada